In [1]:
""" MODEL TRAINING"""

' MODEL TRAINING'

In [13]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score
import joblib


In [14]:
# Load your clean dataset
df = pd.read_csv("battle_ready_countries_final.csv")

# Make sure all columns are numeric
df[['gdp', 'military', 'literacy', 'birth', 'death']] = df[['gdp', 'military', 'literacy', 'birth', 'death']].apply(pd.to_numeric)

# Create pairwise battle data
# For simplicity, generate all possible matchups (excluding self-matchups)
pairs = []
labels = []

for i in range(len(df)):
    for j in range(len(df)):
        if i != j:
            country_a = df.iloc[i]
            country_b = df.iloc[j]

            # Feature: differences between stats
            features = [
                country_a['gdp'] - country_b['gdp'],
                country_a['military'] - country_b['military'],
                country_a['literacy'] - country_b['literacy'],
                country_a['birth'] - country_b['birth'],
                country_a['death'] - country_b['death']
            ]
            pairs.append(features)

            # Label: 1 if country_a wins, else 0 (simplified — assume country with higher GDP+military wins)
            label = int((country_a['gdp'] + country_a['military']) > (country_b['gdp'] + country_b['military']))
            labels.append(label)

X = np.array(pairs)
y = np.array(labels)


In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [16]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train_scaled, y_train)


RandomForestClassifier(random_state=42)

In [17]:
y_pred = model.predict(X_test_scaled)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Accuracy: 1.0

Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      4954
           1       1.00      1.00      1.00      4948

    accuracy                           1.00      9902
   macro avg       1.00      1.00      1.00      9902
weighted avg       1.00      1.00      1.00      9902



In [20]:
import joblib

# Save the trained model and the scaler
joblib.dump(model, "battle_model.pkl")
joblib.dump(scaler, "battle_scaler.pkl")

print("✅ Model and scaler saved successfully!")


✅ Model and scaler saved successfully!
